<center> <b> <font size=5> Differential analysis of count data – the DESeq2 package </font> </b> </center>
<br>
<center> Michael Love$^{1∗}$ , Simon Anders $^2$ , Wolfgang Huber $^2$ </center>
<br>
<center> $^1$Department of Biostatistics, Dana Farber Cancer Institute and
Harvard School of Public Health, Boston, US;
<br>
$^2$European Molecular Biology Laboratory (EMBL), Heidelberg, Germany
<br>
$^∗$ <font face="verdana"> michaelisaiahlove (at) gmail.com</font>
<br> 
<br>
<font size=4> May 1, 2015 </font>
<br>
<br>
** Abstract **
</center>

A basic task in the analysis of count data from RNA-Seq is the detection of differentially expressed
genes. The count data are presented as a table which reports, for each sample, the number of sequence
fragments that have been assigned to each gene. Analogous data also arise for other assay types, including
comparative ChIP-Seq, HiC, shRNA screening, mass spectrometry. An important analysis question is the
quantification and statistical inference of systematic changes between conditions, as compared to within-
condition variability. The package DESeq2 provides methods to test for differential expression by use
of negative binomial generalized linear models; the estimates of dispersion and logarithmic fold changes
incorporate data-driven prior distributions. This vignette explains the use of the package and demonstrates
typical workflows. An RNA-Seq workflow on the Bioconductor website: http://www.bioconductor.org/help/workflows/rnaseqGene/ (formerly the Beginner’s Guide PDF), covers similar material to this
vignette but at a slower pace, including the generation of count matrices from FASTQ files.
**DESeq2 version**: 1.8.1

If you use DESeq2 in published research, please cite:
M. I. Love, W. Huber, S. Anders: **Moderated estimation of
fold change and dispersion for RNA-seq data with DESeq2.**
*Genome Biology* 2014, 15:550.
http://dx.doi.org/10.1186/s13059-014-0550-8

####Saving Session

To save the current R session run the following code:

In [ ]:
save(list = ls(all = TRUE), file= "all.RData")

To load the saved session:

In [ ]:
load("all.RData")

### Contents
* [1 Standard Workflow](#1)
  * [1.1 Quick Start](#1.1)
  * [1.2 Input Data](#1.2)
    * [1.2.1 Why Raw Counts?](#1.2.1)
    * [1.2.2 *SummarizedExperiment* input](#1.2.2)
    * [1.2.3 Count Matrix Input](#1.2.3)
    * [1.2.4 *HTSeq* input](#1.2.4)
    * [1.2.5 Note on Factor Levels](#1.2.5)
    * [1.2.6 Collapsing technical replicates](#1.2.6)
    * [1.2.7 About the pasilla data set](#1.2.7)
  * [1.3 Differential Expression Analysis](#1.3)
  * [1.4 Explring and exporting results](#1.4)
    * [1.4.1 MA-plot](#1.4.1)
    * [1.4.2 Plot counts](#1.4.2)
    * [1.4.3 More informatin on results columns](#1.4.3)
    * [1.4.4 Exporting results to HTML and CSV files](#1.4.4)
  * [1.5 Multi-factor designs](#1.5)
* [2 Data transformations and visualizations](#2)
  * [2.1 Count data transformations](#2.1)
    * [2.1.1 Blind dispersion analysis](#2.1.1)
    * [2.1.2 Extracting transformed values](#2.1.2)
    * [2.1.3 Regularized log transformation](#2.1.3)
    * [2.1.4 Variance stabilizing transformation](#2.1.4)
    * [2.1.5 Effect of transformations on the variance](#2.1.5)
  * [2.2 Data quality assessment by sample clustering and visualization](#2.2)
    * [2.2.1 Heatmap of the count matrix](#2.2.1)
    * [2.2.2 Heatmap of the sample-to-sample distances](#2.2.2)
    * [2.2.3 Principle component plot of the samples](#2.2.3)
* [3 Variations to the standard workflow](#3)
  * [3.1 Wald test individual steps](#3.1)
  * [3.2 Contrasts](#3.2)
  * [3.3 Interactions](#3.3)
  * [3.4 Time-Series experiments](#3.4)
  * [3.5 Likelihood ratio test](#3.5)
  * [3.6 Approach to count outliers](#3.6)
  * [3.7 Dispersion plot and fitting alternatives](#3.7)
    * [3.7.1 Local or mean dispersion fit](#3.7.1)
    * [3.7.2 Supply a custom dispersion fit](#3.7.2)
  * [3.8 Independent filtering of results](#3.8)
  * [3.9 Test of log2 fold change above or below a threshold](#3.9)
  * [3.10 Access to all calculated values](#3.10)
  * [3.11 Sample-/gene-dependent normalization factors](#3.11)
  * [3.12 Model matrix not full-rank](#3.12)
    * [3.12.1 Linear combinations](#3.12.1)
    * [3.12.2 Levels without samples](#3.12.2)
* [4 Theory behind DESeq2](#4)
  * [4.1 The DESeq2 Model](#4.1)
  * [4.2 Changes compared to DESeq package](#4.2)
  * [4.3 Method changes since the 2014 DESeq2 paper](#4.3)
  * [4.4 Count outlier detection](#4.4)
  * [4.5 Contrasts](#4.5)
  * [4.6 Expanded model matrices](#4.6)
  * [4.7 Independent filtering and multiple testing](#4.7)
    * [4.7.1 Filtering criteria](#4.7.1)
    * [4.7.2 Why does it work?](#4.7.2)
    * [4.7.3 Diagnostic plots for multiple testing](#4.7.3)
* [5 Frequently Asked Questions](#5)
  * [5.1 How can I get support for DESeq2?](#5.1)
  * [5.2 Why are some p values set to NA?](#5.2)
  * [5.3 How can I get unfiltered DESeq results?](#5.3)
  * [5.4 How do I use the variance stabilized or rlog transformed data for differential testing?](#5.4)
  * [5.5 Can I use DESeq2 to analyze paired samples?](#5.5)
  * [5.6 Can I use DESeq2 to contrast the levels of 100 groups?](#5.6)
  * [5.7 Can I use DESeq2 to analyze a data set without replicates?](#5.7)
  * [5.8 How can I include a continuous covariate in the design formula?](#5.8)
  * [5.9 What are the exact steps performed by the DESeq()?](#5.9)
* [6 Acknowledgements](#6)
* [References](#ref)


<a id="1"></a>
## 1. Standard workflow
<a id="1.1"></a>
### 1.1 Quick start
Here we show the most basic steps for a differential expression analysis. These steps imply you have a
*SummarizedExperiment* object <font face="courier">se</font> with columns <font face="courier">batch</font> and <font face="courier">condition</font> in <font face="courier">colData(se)</font>, and that condition
has levels “untreat” and “treat”.
```r
dds <- DESeqDataSet(se, ~ batch + condition)
dds <- DESeq(dds)
res <- results(dds, contrast=c("condition","treat","untreat"))
```
If you have a count matrix and sample information table, the first line would instead use <font face="courier">DESeqDataSetFromMatrix</font>,
shown below.
<a id="1.2"></a>
### 1.2 Input data
<a id="1.2.1"></a>
#### 1.2.1 Why raw counts?
As input, the *DESeq2* package expects count data as obtained, e. g., from RNA-Seq or another high-throughput
sequencing experiment, in the form of a matrix of integer values. The value in the $i$-th row and the $j$-th column
of the matrix tells how many reads have been mapped to gene $i$ in sample $j$. Analogously, for other types of
assays, the rows of the matrix might correspond e. g. to binding regions (with ChIP-Seq) or peptide sequences
(with quantitative mass spectrometry).

The count values must be raw counts of sequencing reads. This is important for *DESeq2*’s statistical model
[1] to hold, as only the actual counts allow assessing the measurement precision correctly. Hence, please do
not supply other quantities, such as (rounded) normalized counts, or counts of covered base pairs – this will
only lead to nonsensical results.
<a id="1.2.2"></a>
#### 1.2.2 SummarizedExperiment input
The class used by the *DESeq2* package to store the read counts is *DESeqDataSet* which extends the *SummarizedExperiment* class of the [**GenomicRanges**](http://bioconductor.org/packages/release/bioc/html/GenomicRanges.html) package. This facilitates preparation steps and also downstream
exploration of results. For counting aligned reads in genes, the <font face="courier">summarizeOverlaps</font> function of [**GenomicAlignments**](http://bioconductor.org/packages/release/bioc/html/GenomicAlignments.html) with <font face="courier">mode="Union"</font> is encouraged, resulting in a *SummarizedExperiment* object.

An example of the steps to produce a *SummarizedExperiment* can be found in an RNA-Seq workflow on the
Bioconductor website: http://www.bioconductor.org/help/workflows/rnaseqGene/ and in the vignette
for the data package airway . Here we load the *SummarizedExperiment* from that package in order to build a
*DESeqDataSet*.

In [ ]:
library("airway")
data("airway")
se <- airway

A *DESeqDataSet* object must have an associated design formula. The design formula expresses the variables
which will be used in modeling. The formula should be a tilde (∼) followed by the variables with plus signs between them (it will be coerced into an formula if it is not already). An intercept is included, representing
the base mean of counts. The design can be changed later, however then all differential analysis steps
should be repeated, as the design formula is used to estimate the dispersions and to estimate the log2 fold
changes of the model. The constructor function below shows the generation of a *DESeqDataSet* from a
*SummarizedExperiment* <font face="courier"> se </font>.

*Note*: In order to benefit from the default settings of the package, you should put the variable of interest at
the end of the formula and make sure the control level is the first level.

In [ ]:
library("DESeq2")
ddsSE <- DESeqDataSet(se, design = ~ cell + dex)
ddsSE

<a id="1.2.3"></a>
#### 1.2.3 Count matrix input
Alternatively, the function <font face="courier">DESeqDataSetFromMatrix</font> can be used if you already have a matrix of read counts
prepared. For this function you should provide the counts matrix, the column information as a *DataFrame* or
*data.frame* and the design formula. To demonstate the use of this function, we first load the <font face="courier">pasillaGenes</font>
data object, to pull out the count matrix, <font face="courier">countData</font>, and sample information, <font face="courier">colData</font>.

In [ ]:
library("pasilla")
library("Biobase")
data("pasillaGenes")
countData <- counts(pasillaGenes)
colData <- pData(pasillaGenes)[,c("condition","type")]

<font face="courier">countData</font> should be a matrix of read counts, where columns correspond to different samples. <font face="courier">colData</font> should
be a *data.frame* or *DataFrame* of information about the samples, where the rows directly match the columns
of <font face="courier">countData</font>.

In [ ]:
head(countData)

In [ ]:
head(colData)

These two objects can be read into R from flat files using base R functions such as <font face="courier">read.csv</font> or <font face="courier">read.delim</font>.
For HTSeq count files, see the dedicated input function below. If you have used the <font face="courier">featureCounts</font> function
in the [**Rsubread**](http://bioconductor.org/packages/release/bioc/html/Rsubread.html) package, the matrix of read counts can be directly provided from the <font face="courier">"counts"</font> element in
the list output. With the count matrix, <font face="courier">countData</font>, and the sample information,<font face="courier"> colData</font>, we can construct
a *DESeqDataSet*:

In [ ]:
dds <- DESeqDataSetFromMatrix(countData = countData,
                              colData = colData,
                              design = ~ condition)
dds

If you have additional feature data, it can be added to the *DESeqDataSet* by adding to the metadata columns
of a newly constructed object. (Here we add redundant data just for demonstration, as the gene names are
already the rownames of the dds.)

In [ ]:
featureData <- data.frame(gene=rownames(pasillaGenes))
(mcols(dds) <- DataFrame(mcols(dds), featureData))

<a id="1.2.4"></a>
#### 1.2.4 HTSeq input
You can use the function <font face="courier">DESeqDataSetFromHTSeqCount</font> if you have htseq-count from the [**HTSeq python package**](http://www-huber.embl.de/users/anders/HTSeq/doc/overview.html) . For an example of using the python scripts, see the [**pasilla**](http://bioconductor.org/packages/release/data/experiment/html/pasilla.html) data package. First you will want to
specify a variable which points to the directory in which the HTSeq output files are located.

In [ ]:
directory <- "/path/to/your/files/"

However, for demonstration purposes only, the following line of code points to the directory for the demo
HTSeq output files packages for the [**pasilla**](http://bioconductor.org/packages/release/data/experiment/html/pasilla.html) package.

In [ ]:
directory <- system.file("extdata", package="pasilla", mustWork=TRUE)

We specify which files to read in using <font face="courier">list.files</font>, and select those files which contain the string "treated"
using <font face="courier">grep</font>. The sub function is used to chop up the sample filename to obtain the condition status, or you
might alternatively read in a phenotypic table using <font face="courier">read.table </font>.

In [ ]:
sampleFiles <- grep("treated",list.files(directory),value=TRUE)
sampleCondition <- sub("(.*treated).*","\\1",sampleFiles)
sampleTable <- data.frame(sampleName = sampleFiles,
                          fileName = sampleFiles,
                          condition = sampleCondition)
ddsHTSeq <- DESeqDataSetFromHTSeqCount(sampleTable = sampleTable,
                                       directory = directory,
                                       design= ~ condition)
ddsHTSeq

<a id="1.2.5"></a>
#### 1.2.5 Note on factor levels
In the three examples above, we applied the function <font face="courier">factor</font> to the column of interest in <font face="courier">colData</font>, supplying
a character vector of levels. It is important to supply levels (otherwise the levels are chosen in alphabetical
order) and to put the “control” or “untreated” level as the first element (”reference level”), so that the
log2 fold changes produced by default will be the expected comparison against the reference level, that is
$\log_2 (\mathrm{treated}/\mathrm{untreated})$. An R function for easily changing the reference level is <font face="courier">relevel</font>. An example of
setting the reference level of a factor with <font face="courier">relevel</font> is:

In [ ]:
dds$condition <- relevel(dds$condition, "untreated")

In addition, when subsetting the columns of a *DESeqDataSet*, i.e., when removing certain samples from the
analysis, it is possible that all the samples for one or more levels of a variable in the design formula are removed.
In this case, the droplevels function can be used to remove those levels which do not have samples in the
current *DESeqDataSet*:

In [ ]:
dds$condition <- droplevels(dds$condition)

<a id="1.2.6"></a>
#### 1.2.6 Collapsing technical replicates
*DESeq2* provides a function <font face="courier">collapseReplicates</font> which can assist in combining the counts from technical
replicates into single columns. See the manual page for an example of the use of <font face="courier">collapseReplicates</font>.
<a id="1.2.7"></a>
#### 1.2.7 About the pasilla dataset
We continue with the [**pasilla**](http://bioconductor.org/packages/release/data/experiment/html/pasilla.html) data constructed from the count matrix method above. This data set is from
an experiment on *Drosophila melanogaster* cell cultures and investigated the effect of RNAi knock-down of
the splicing factor *pasilla* [3]. The detailed transcript of the production of the pasilla data is provided in the
vignette of the data package [**pasilla**](http://bioconductor.org/packages/release/data/experiment/html/pasilla.html).
<a id="1.3"></a>
### 1.3 Differential expression analysis
The standard differential expression analysis steps are wrapped into a single function, <font face="courier">DESeq</font>. The steps of this
function are described in Section 4.1 and in the manual page for <font face="courier">?DESeq</font>. The individual sub-functions which
are called by <font face="courier">DESeq</font> are still available, described in Section 3.1.

Results tables are generated using the function <font face="courier">results</font>, which extracts a results table with log2 fold changes,
$p$ values and adjusted $p$ values. With no arguments to <font face="courier">results</font>, the results will be for the last variable in the
design formula, and if this is a factor, the comparison will be the last level of this variable over the first level.
Details about the comparison are printed to the console. The text, condition treated vs untreated, tells
you that the estimates are of the logarithmic fold change $\log_2 (\mathrm{treated}/\mathrm{untreated})$.

In [ ]:
dds <- DESeq(dds)
res <- results(dds)
res

These steps should take less than 30 seconds for most analyses. For experiments with many samples (e.g. 100
samples), one can take advantage of parallelized computation. Both of the above functions have an argument
<font face="courier">parallel</font> which if set to <font face="courier">TRUE</font> can be used to distribute computation across cores specified by the register
function of [**BiocParallel**](http://bioconductor.org/packages/release/bioc/html/BiocParallel.html). For example, the following chunk (not evaluated here), would register 4 cores, and
then the two functions above, with <font face="courier">parallel=TRUE</font>, would split computation over these cores.

In [ ]:
library("BiocParallel")
register(MulticoreParam(4))

We can order our results table by the smallest adjusted $p$ value:

In [ ]:
resOrdered <- res[order(res$padj),]
head(resOrdered)

We can summarize some basic tallies using the <font face="courier">summary</font> function.

In [ ]:
summary(res)

The <font face='courier'> results</font> function contains a number of arguments to customize the results table which is generated.
Note that the <font face='courier'>results</font> function automatically performs independent filtering based on the mean of normalized
counts for each gene, optimizing the number of genes which will have an adjusted $p$ value below a given FDR
cutoff, <font face='courier'>alpha</font>, By default the argument <font face='courier'>alpha</font> is set to 0.1. If the adjusted $p$ value cutoff will be a value other
than 0.1, <font face='courier'>alpha</font> should be set to that value. Independent filtering will be discussed further in Section 3.8.

If a multi-factor design is used, or if the variable in the design formula has more than two levels, the <font face='courier'>contrast</font>
argument of <font face='courier'>results</font> can be used to extract different comparisons from the *DESeqDataSet* returned by <font face='courier'>DESeq</font>.
Multi-factor designs are discussed further in Section 1.5, and the use of the <font face='courier'>contrast</font> argument is dicussed in
Section 3.2.

For advanced users, note that all the values calculated by the *DESeq2* package are stored in the *DESeqDataSet*
object, and access to these values is discussed in Section 3.10.
<a id="1.4"></a>
### 1.4 Exploring and exporting results
<a id="1.4.1"></a>
#### 1.4.1 MA-plot
In *DESeq2* , the function <font face='courier'>plotMA</font> shows the log2 fold changes attributable to a given variable over the mean
of normalized counts. Points will be colored red if the adjusted $p$ value is less than 0.1. Points which fall out
of the window are plotted as open triangles pointing either up or down.

In [ ]:
plotMA(res, main="DESeq2", ylim=c(-2,2))
identify(res$baseMean, res$log2FoldChange)

The MA-plot of log2 fold changes returned by *DESeq2* allows us to see how the shrinkage of fold changes
works for genes with low counts. After calling <font face='courier'>plotMA</font>, one can use the function <font face='courier'>identify</font> to interactively detect the row number of individual
genes by clicking on the plot.


You can still obtain results tables which include the “unshrunken” log2 fold
changes (for a simple comparison, the ratio of the mean normalized counts in the two groups). A column
<font face='courier'>lfcMLE</font> with the unshrunken maximum likelihood estimate (MLE) for the log2 fold change will be added with
an additional argument to <font face='courier'>results</font>:

In [ ]:
resMLE <- results(dds, addMLE=TRUE)
head(resMLE, 4)

<a id="1.4.2"></a>
#### 1.4.2 Plot counts
It can also be useful to examine the counts of reads for a single gene across the groups. A simple function
for making this plot is <font face='courier'>plotCounts</font>, which normalizes counts by sequencing depth and adds a pseudocount
of $\frac{1}{2}$ to allow for log scale plotting. The counts are grouped by the variables in <font face='courier'>intgroup</font>, where more than
one variable can be specified. Here we specify the gene which had the smallest $p$ value from the results table
created above. You can select the gene to plot by rowname or by numeric index.

In [ ]:
plotCounts(dds, gene=which.min(res$padj), intgroup="condition")

For customized plotting, an argument <font face='courier'>returnData</font> specifies that the function should only return a *data.frame*
for plotting with <font face='courier'>ggplot</font>.

Figure 2: Plot of counts for one gene. The plot of normalized counts (plus a pseudocount of 2 1 ) either
made using the plotCounts function (left) or using another plotting library (right, using ggplot2 ).
*insert figure here*


In [ ]:
d <- plotCounts(dds, gene=which.min(res$padj), intgroup="condition",
                returnData=TRUE)
library("ggplot2")
ggplot(d, aes(x=condition, y=count)) +
geom_point(position=position_jitter(w=0.1,h=0)) +
scale_y_log10(breaks=c(25,100,400))

<a id="1.4.3"></a>
#### 1.4.3 More information on results columns
Information about which variables and tests were used can be found by calling the function <font face='courier'>mcols</font> on the
results object.

In [ ]:
mcols(res)$description

For a particular gene, a log2 fold change of −1 for <font face='courier'>condition treated vs untreated</font> means that the
treatment induces a change in observed expression level of $2^{−1} = 0.5$ compared to the untreated condition. If
the variable of interest is continuous-valued, then the reported log2 fold change is per unit of change of that
variable.

Note: some values in the results table can be set to <font face='courier'>NA</font>, for either one of the following reasons:
1. If within a row, all samples have zero counts, the baseMean column will be zero, and the log2 fold
change estimates, $p$ value and adjusted $p$ value will all be set to <font face='courier'>NA</font>.
2. If a row contains a sample with an extreme count outlier then the $p$ value and adjusted $p$ value are set
to <font face='courier'>NA</font>. These outlier counts are detected by Cook’s distance. Customization of this outlier filtering and
description of functionality for replacement of outlier counts and refitting is described in Section 3.6,
3. If a row is filtered by automatic independent filtering, based on low mean normalized count, then only
the adjusted $p$ value is set to <font face='courier'>NA</font>. Description and customization of independent filtering is described in
Section 3.8.

The column of <font face='courier'>log2FoldChange</font> for the default workflow will contain shrunken estimates of fold change as
mentioned above. It is possible to add a column to the results table – without rerunning the analysis – which
contains the unshrunken, or maximum likelihood estimates (MLE), log2 fold changes. This will add the column
<font face='courier'>lfcMLE</font> directly after <font face='courier'>log2FoldChange</font>.

In [ ]:
head(results(dds, addMLE=TRUE),4)

<a id="1.4.4"></a>
#### 1.4.4 Exporting results to HTML or CSV files
An HTML report of the results with plots and sortable/filterable columns can be exported using the [**Report-
ingTools**](http://bioconductor.org/packages/release/bioc/html/ReportingTools.html) package on a *DESeqDataSet* that has been processed by the <font face='courier'>DESeq</font> function. For a code example,
see the “RNA-seq differential expression” vignette at the [**ReportingTools**](http://bioconductor.org/packages/release/bioc/html/ReportingTools.html) page, or the manual page for the
<font face='courier'>publish</font> method for the *DESeqDataSet* class.

A plain-text file of the results can be exported using the base R functions <font face='courier'>write.csv</font> or <font face='courier'>write.delim</font>. We
suggest using a descriptive file name indicating the variable and levels which were tested.

In [ ]:
write.csv(as.data.frame(resOrdered),
          file="condition_treated_results.csv")

Exporting only the results which pass an adjusted $p$ value threshold can be accomplished with the <font face='courier'>subset</font>
function, followed by the <font face='courier'>write.csv</font> function.

In [ ]:
resSig <- subset(resOrdered, padj < 0.1)
resSig

<a id="1.5"></a>
### 1.5 Multi-factor designs
Experiments with more than one factor influencing the counts can be analyzed using model formula including
the additional variables. The data in the [**pasilla**](http://bioconductor.org/packages/release/data/experiment/html/pasilla.html) package have a condition of interest (the column <font face='courier'>condition</font>),
as well as information on the type of sequencing which was performed (the column <font face='courier'>type</font>), as we can see below:

In [ ]:
colData(dds)

We create a copy of the *DESeqDataSet*, so that we can rerun the analysis using a multi-factor design.

In [ ]:
ddsMF <- dds

We can account for the different types of sequencing, and get a clearer picture of the differences attributable
to the treatment. As <font face='courier'>condition</font> is the variable of interest, we put it at the end of the formula. Thus the
<font face='courier'>results</font> function will by default pull the <font face='courier'>condition</font> results unless <font face='courier'>contrast</font> or <font face='courier'>name</font> arguments are specified.
Then we can re-run <font face='courier'>DESeq</font>:

In [ ]:
design(ddsMF) <- formula(~ type + condition)
ddsMF <- DESeq(ddsMF)

Again, we access the results using the <font face='courier'>results</font> function.

In [ ]:
resMF <- results(ddsMF)
head(resMF)

It is also possible to retrieve the log2 fold changes, $p$ values and adjusted $p$ values of the <font face='courier'>type</font> variable.
The <font face='courier'>contrast</font> argument of the function <font face='courier'>results</font> takes a character vector of length three: the name of the
variable, the name of the factor level for the numerator of the log2 ratio, and the name of the factor level
for the denominator. The <font face='courier'>contrast</font> argument can also take other forms, as described in the help page for <font face='courier'>results</font> and in Section 3.2.

In [ ]:
resMFType <- results(ddsMF, contrast=c("type","single-read","paired-end"))
head(resMFType)

If the variable is continuous or an interaction term (see Section 3.3) then the results can be extracted using
the <font face='courier'>name</font> argument to <font face='courier'>results</font>, where the name is one of elements returned by <font face='courier'>resultsNames(dds)</font>.
<a id="2"></a>
## 2 Data transformations and visualization
<a id="2.1"></a>
### 2.1 Count data transformations
In order to test for differential expression, we operate on raw counts and use discrete distributions as described
in the previous Section 1.3. However for other downstream analyses – e.g. for visualization or clustering – it
might be useful to work with transformed versions of the count data.

Maybe the most obvious choice of transformation is the logarithm. Since count values for a gene can be zero
in some conditions (and non-zero in others), some advocate the use of *pseudocounts*, i. e. transformations of
the form
$$y=\log_2(n+1)\:\mathrm{or}\,\,\mathrm{more}\,\,\mathrm{generally},\:y=\log_2(n+n_0),$$
where $n$ represents the count values and $n_0$ is a positive constant.

In this section, we discuss two alternative approaches that offer more theoretical justification and a rational
way of choosing the parameter equivalent to $n_0$ above. The *regularized logarithm* or $\mathrm{rlog}$ incorporates a prior
on the sample differences [1], and the other uses the concept of variance stabilizing transformations (VST)
[4, 5, 6]. Both transformations produce transformed data on the $\log_2$ scale which has been normalized with
respect to library size.

The point of these two transformations, the $\mathrm{rlog}$ and the VST, is to remove the dependence of the variance
on the mean, particularly the high variance of the logarithm of count data when the mean is low. Both $\mathrm{rlog}$
and VST use the experiment-wide trend of variance over mean, in order to transform the data to remove
the experiment-wide trend. Note that we do not require or desire that all the genes have exactly the same
variance after transformation. Indeed, in Figure 4 below, you will see that after the transformations the genes
with the same mean do not have exactly the same standard deviations, but that the experiment-wide trend
has flattened. It is those genes with row variance above the trend which will allow us to cluster samples into
interesting groups.
<a id="2.1.1"></a>
#### 2.1.1 Blind dispersion estimation
The two functions, <font face='courier'>rlog</font> and <font face='courier'>varianceStabilizingTransformation</font>, have an argument <font face='courier'>blind</font>, for whether
the transformation should be blind to the sample information specified by the design formula. When <font face='courier'>blind</font>
equals <font face='courier'>TRUE</font> (the default), the functions will re-estimate the dispersions using only an intercept (design formula
∼ 1). This setting should be used in order to compare samples in a manner wholly unbiased by the information
about experimental groups, for example to perform sample QA (quality assurance) as demonstrated below.

However, blind dispersion estimation is not the appropriate choice if one expects that many or the majority of
genes (rows) will have large differences in counts which are explanable by the experimental design, and one
wishes to tranform the data for downstream analysis. In this case, using blind dispersion estimation will lead
to large estimates of dispersion, as it attributes differences due to experimental design as unwanted “noise”,
and shrinks the tranformed values towards each other. By setting <font face='courier'>blind</font> to <font face='courier'>FALSE</font>, the dispersions already
estimated will be used to perform transformations, or if not present, they will be estimated using the current
design formula. Note that only the fitted dispersion estimates from mean-dispersion trend line is used in the
transformation. So setting <font face='courier'>blind</font> to <font face='courier'>FALSE</font> is still mostly unbiased by the information about the samples.
<a id="2.1.2"></a>
#### 2.1.2 Extracting transformed values
The two functions return an object of class *DESeqTransform* which is a subclass of *SummarizedExperiment*.
The <font face='courier'>assay</font> function is used to extract the matrix of normalized values:

In [ ]:
rld <- rlog(dds)
vsd <- varianceStabilizingTransformation(dds)
rlogMat <- assay(rld)
vstMat <- assay(vsd)

Note that if you have many samples, and the <font face='courier'>rlog</font> function is taking too long, there is an argument <font face='courier'>fast=TRUE</font>,
which will perform an approximation of the rlog: instead of shrinking the samples independently, the function
will find the optimal amount of shrinkage for each gene given the mean counts. The optimization is performed
on the same likelihood of the data as the original <font face='courier'>rlog</font>. The speed-up for a dataset with 100 samples is around 15x.
<a id="2.1.3"></a>
#### 2.1.3 Regularized log transformation
The function <font face='courier'>rlog</font>, stands for *regularized log*, transforming the original count data to the log2 scale by fitting
a model with a term for each sample and a prior distribution on the coefficients which is estimated from the
data. This is the same kind of shrinkage (sometimes referred to as regularization, or moderation) of log fold
changes used by the <font face='courier'>DESeq</font> and <font face='courier'>nbinomWaldTest</font>, as seen in Figure 1. The resulting data contains elements
defined as:
$$\log_2(q_{ij})=\beta_{i0}+\beta_{ij}$$

where $q_{ij}$ is a parameter proportional to the expected true concentration of fragments for gene $i$ and sample
$j$ (see Section 4.1), $\beta_{i0}$ is an intercept which does not undergo shrinkage, and $\beta_{ij}$ is the sample-specific effect
which is shrunk toward zero based on the dispersion-mean trend over the entire dataset. The trend typically
captures high dispersions for low counts, and therefore these genes exhibit higher shrinkage from the rlog.

Note that, as $q_{ij}$ represents the part of the mean value $\mu_{ij}$ after the size factor $s_j$ has been divided out, it
is clear that the rlog transformation inherently accounts for differences in sequencing depth. Without priors,
this design matrix would lead to a non-unique solution, however the addition of a prior on non-intercept betas
allows for a unique solution to be found. The regularized log transformation is preferable to the variance
stabilizing transformation if the size factors vary widely.
<a id="2.1.4"></a>
#### 2.1.4 Variance stabilizing transformation
Above, we used a parametric fit for the dispersion. In this case, the closed-form expression for the variance stabi-
lizing transformation is used by <font face='courier'>varianceStabilizingTransformation</font>, which is derived in the file <font face='courier'>vst.pdf</font>,
that is distributed in the package alongside this vignette. If a local fit is used (option <font face='courier'>fitType="locfit"</font> to <font face='courier'>
estimateDispersions</font>) a numerical integration is used instead.

The resulting variance stabilizing transformation is shown in Figure 3. The code that produces the figure is
hidden from this vignette for the sake of brevity, but can be seen in the <font face='courier'>.Rnw</font> or <font face='courier'>.R</font> source file. Note that the
vertical axis in such plots is the square root of the variance over all samples, so including the variance due to
the experimental conditions. While a flat curve of the square root of variance over the mean may seem like the goal of such transformations, this may be unreasonable in the case of datasets with many true differences
due to the experimental conditions.

Figure 3: VST and log2. Graphs of the variance stabilizing transformation for sample 1, in blue, and of the
transformation f (n) = log 2 (n/s 1 ), in black. n are the counts and s 1 is the size factor for the first sample.

Figure 4: Standard deviation over mean. Per-gene standard deviation (taken across samples), against the
rank of the mean, for the shifted logarithm log 2 (n + 1) (left), the regularized log transformation (center) and
the variance stabilizing transformation (right)
<a id="2.1.5"></a>
#### 2.1.5 Effects of transformations on the variance
Figure 4 plots the standard deviation of the transformed data, across samples, against the mean, using the
shifted logarithm transformation (1), the regularized log transformation and the variance stabilizing transfor-
mation. The shifted logarithm has elevated standard deviation in the lower count range, and the regularized
log to a lesser extent, while for the variance stabilized data the standard deviation is roughly constant along
the whole dynamic range.

In [ ]:
library("vsn")
par(mfrow=c(1,3))
notAllZero <- (rowSums(counts(dds))>0)
meanSdPlot(log2(counts(dds,normalized=TRUE)[notAllZero,] + 1))
meanSdPlot(assay(rld[notAllZero,]))
meanSdPlot(assay(vsd[notAllZero,]))

<a id="2.2"></a>
### 2.2 Data quality assessment by sample clustering and visualization
Data quality assessment and quality control (i. e. the removal of insufficiently good data) are essential steps
of any data analysis. These steps should typically be performed very early in the analysis of a new data set,
preceding or in parallel to the differential expression testing.

We define the term quality as [*fitness for purpose*](https://en.wikipedia.org/wiki/Quality_%28business%29). Our purpose is the detection of differentially expressed
genes, and we are looking in particular for samples whose experimental treatment suffered from an anormality
that renders the data points obtained from these particular samples detrimental to our purpose.
<a id="2.2.1"></a>
#### 2.2.1 Heatmap of the count matrix
To explore a count matrix, it is often instructive to look at it as a heatmap. Below we show how to produce
such a heatmap from the raw and transformed data.

In [ ]:
library("RColorBrewer")
library("gplots")
select <- order(rowMeans(counts(dds,normalized=TRUE)),decreasing=TRUE)[1:30]
hmcol <- colorRampPalette(brewer.pal(9, "GnBu"))(100)
heatmap.2(counts(dds,normalized=TRUE)[select,], col = hmcol,
          Rowv = FALSE, Colv = FALSE, scale="none",
          dendrogram="none", trace="none", margin=c(10,6))
heatmap.2(assay(rld)[select,], col = hmcol,
          Rowv = FALSE, Colv = FALSE, scale="none",
          dendrogram="none", trace="none", margin=c(10, 6))
heatmap.2(assay(vsd)[select,], col = hmcol,
          Rowv = FALSE, Colv = FALSE, scale="none",
          dendrogram="none", trace="none", margin=c(10, 6))

Figure 5: Heatmaps showing the expression data of the 30 most highly expressed genes. The data is of raw
counts (left), from regularized log transformation (center) and from variance stabilizing transformation (right).

Figure 6: Sample-to-sample distances. Heatmap showing the Euclidean distances between the samples as
calculated from the regularized log transformation.

Figure 7: PCA plot. PCA plot. The 7 samples shown in the 2D plane spanned by their first two principal
components. This type of plot is useful for visualizing the overall effect of experimental covariates and batch
effects.
<a id="2.2.2"></a>
#### 2.2.2 Heatmap of the sample-to-sample distances
Another use of the transformed data is sample clustering. Here, we apply the <font face='courier'>dist</font> function to the transpose
of the transformed count matrix to get sample-to-sample distances. We could alternatively use the variance
stabilized transformation here.

In [ ]:
distsRL <- dist(t(assay(rld)))

A heatmap of this distance matrix gives us an overview over similarities and dissimilarities between samples
(Figure 6): We have to provide a hierarchical clustering <font face='courier'>hc</font> to the <font face='courier'>heatmap.2</font> function based on the sample
distances, or else the <font face='courier'>heatmap.2</font> function would calculate a clustering based on the distances between the
rows/columns of the distance matrix.

In [ ]:
mat <- as.matrix(distsRL)
rownames(mat) <- colnames(mat) <- with(colData(dds),
                                       paste(condition, type, sep=" : "))
hc <- hclust(distsRL)
heatmap.2(mat, Rowv=as.dendrogram(hc),
          symm=TRUE, trace="none",
          col = rev(hmcol), margin=c(13, 13))

<a id="2.2.3"></a>
#### 2.2.3 Principal component plot of the samples
Related to the distance matrix of Section 2.2.2 is the PCA plot of the samples, which we obtain as follows
(Figure 7).

In [ ]:
plotPCA(rld, intgroup=c("condition", "type"))

Figure 8: PCA plot. PCA plot customized using the ggplot2 library.

It is also possible to customize the PCA plot using the <font face='courier'>ggplot</font> function.

In [ ]:
data <- plotPCA(rld, intgroup=c("condition", "type"), returnData=TRUE)
percentVar <- round(100 * attr(data, "percentVar"))
ggplot(data, aes(PC1, PC2, color=condition, shape=type)) +
geom_point(size=3) +
xlab(paste0("PC1: ",percentVar[1],"% variance")) +
ylab(paste0("PC2: ",percentVar[2],"% variance"))

<a id="3"></a>
## 3 Variations to the standard workflow
<a id="3.1"></a>
### 3.1 Wald test individual steps
The function <font face='courier'>DESeq</font> runs the following functions in order:

In [ ]:
dds <- estimateSizeFactors(dds)
dds <- estimateDispersions(dds)
dds <- nbinomWaldTest(dds)

<a id="3.2"></a>
### 3.2 Contrasts
A contrast is a linear combination of estimated log2 fold changes, which can be used to test if differences
between groups are equal to zero. The simplest use case for contrasts is an experimental design containing
a factor with three levels, say A, B and C. Contrasts enable the user to generate results for all 3 possible
differences: log2 fold change of B vs A, of C vs A, and of C vs B (the other three possible pairs will simply
have −1x the log2 fold changes of these three).

In order to fit models with “shrunken” log2 fold changes in a manner which is independent to the choice of
reference level, *DESeq2* uses “expanded model matrices”, described further in Section 4.6. The expanded
model matrices include a coefficient for each level of the factors in addition to an intercept. The <font face='courier'>contrast</font>
argument of <font face='courier'>results</font> function is again used to extract test results of log2 fold changes of interest.

Log2 fold changes can also be added and subtracted by providing a <font face='courier'>list</font> to the <font face='courier'>contrast</font> argument with two
elements: the names of the log2 fold changes to add, and the names of the log2 fold changes to subtract.
Alternatively, a numeric vector of the length of <font face='courier'>resultsNames(dds)</font> can be provided, for manually specifying
the linear combination of terms. Demonstrations of the use of contrasts for various designs can be found in
the examples section of the help page for the <font face='courier'>results</font> function. The formula that is used to generate the
contrasts can be found in Section 4.5.
<a id="3.3"></a>
#### 3.3 Interactions
Interaction terms can be added to the design formula, in order to test, for example, if the log2 fold change
attributable to a given condition is different based on a second variable, for example if the treatment effect
of a drug is differs based on another grouping variable like species. Interactions are specified in R formula
using a colon, :, between the two variables names. Demonstrations of extracting results for many different
kinds of interaction model are shown in the examples section of the help page for <font face='courier'>results</font>. When interations
are present in the design, and a prior is used for the log2 fold changes, then only the interaction terms are
moderated, and the main effect terms are not moderated. See <font face='courier'>?nbinomWaldTest</font> for more details.

Note that if the comparisons of interest are, for example, the effect of a condition in different sets of samples,
many users find that a simpler design which likewise allows for a set-specific condition effect is the following:
first combine the factors of interest into a single factor with all combinations of the original factors, and then
use a design with just this factor. For example, if we have two factors <font face='courier'>set</font> and <font face='courier'>condition</font>, and we want to
test for the set-specific condition effects in all sets:

In [ ]:
dds$group <- factor(paste0(dds$set, dds$condition))
design(dds) <- ~ group
dds <- DESeq(dds)
resultsNames(dds)
results(dds, contrast=c("group", "treated", "untreated"))

If one wants to test whether the condition effect is *different* across sets, then a design with interactions makes
these comparisons easier to test. It is important to remember that interactions are *additional terms* (addition
in the log scale), beyond the *main effect* terms. If the *main effect* of a condition on counts is a log2 fold
change of 2, and the *interaction term* for the condition in a particular set Y is a log2 fold change of 0.5, then
the log2 fold change for the condition for set Y is 2 + 0.5 = 2.5. Testing the interation term alone tests if
the condition effect in set Y is different than in the other set(s). Testing the *sum* of the main effect and the
interaction term tests if there is a condition effect in set Y. This is shown in more detail in the examples section
of the help page for <font face='courier'>results</font>.

Designs with factors with only two levels and an interaction term present a special case to the normal *DESeq2*
methods, because we change the type of model matrix used in the fitting, using what we call R’s “standard
model matrix”. For all other designs, we use an “expanded model matrix” which fits a term for each level in
the model, so there are no reference levels. The reason to use a standard model matrix for this special case
of factors with two levels and an interaction term is that it is desirable to have only a single term to test for
a significant interaction between the variables. Furthermore, the expanded design matrices are not necessary
in this case for their purpose (for other designs, they provide symmetric shrinkage regardless of the reference
level).

This does create a difference in the interpretation of the contrasts extracted with <font face='courier'>results</font> for designs with
interactions: for expanded design matrices, the main effect is an average effect across all sets; while for standard
design matrices, the main effect is only for the reference level of the other factors. We explain in more detail
below:

Suppose we have an interaction design of <font face='courier'> ∼ set + condition + set:condition </font>.

**Expanded model matrices** The following code produces a comparison of B over A *across all sets*.

```r
results(dds, contrast=c("condition","B","A"))
```

**Standard model matrices** The above results table now produces a comparison of B over A only for the
*reference level* of set. Let us suppose the reference level of set is X, and the second set is Y. So for standard
model matrices, the above results table gives the condition effect for set X.

Below we explain how to interpret the interaction terms for both kinds of model matrix.

**Expanded model matrices** The following code would give the *interaction effect* for condition in set Y, which
tests if the condition effect is different in set Y than in the other sets.

```r
results(dds, contrast=list("setY.conditionB","setY.conditionA"))
```

And the following would give the condition effect in set Y (the main effect and the interaction terms added
together):

```r
results(dds, contrast=list(c("conditionB","setY.conditionB"),
                           c("conditionA","setY.conditionA")))
```

**Standard model matrices** Here, we have only a single interaction term, <font face='courier'>setY.conditionB</font>. This term
represents the additional effect of condition for set Y compared to the main effect of condition in set X. By
symmetry, it also represents the additional effect of set in condition B compared to the main effect of set in condition A. Testing if the condition effect is different between the two sets is accomplished by extracting only
this interaction term:

```r
results(dds, name="setY.conditionB")
```

The effect of condition in set Y is the above interaction term added to the main effect of condition B vs A in
set X:

```r
results(dds, contrast=list(c("conditionB","setY.conditionB")))
```

For standard model matrices, to obtain the “average condition effect” for both sets, we can use a numeric
contrast, assigning a 1 to the main effect <font face='courier'>conditionB</font> and a 0.5 to the interaction effect <font face='courier'>setY.conditionB</font>,
as they appear in <font face='courier'>resultsNames(dds)</font>. This can be understood as taking us halfway from the condition effect
in set X to the condition effect in set Y:

```r
resultsNames(dds)
results(dds, contrast=c(0,0,1,0.5))
```

You can use the following lines of code to find out the model matrix type and to extract the model matrix
which was used by DESeq. For models with shrinkage on log2 fold changes (<font face='courier'>betaPrior=TRUE</font>), the type will
be <font face='courier'>"expanded"</font> unless the design has factors with only two levels and an interaction term is included, in which
case it will be "<font face='courier'>standard</font>".

In [ ]:
attr(dds, "modelMatrixType")
attr(dds, "modelMatrix")

<a id="3.4"></a>
### 3.4 Time-series experiments
There are a number of ways to analyze time-series experiments, depending on the biological question of
interest. In order to test for any differences over multiple time points, once can use a design including the
time factor, and then test using the likelihood ratio test as described in Section 3.5, where the time factor
is removed in the reduced formula. For a control and treatment time series, one can use a design formula
containing the condition factor, the time factor, and the interaction of the two. In this case, using the
likelihood ratio test with a reduced model which does not contain the interaction terms will test whether
the condition induces a change in gene expression at any time point after the reference level time point
(time 0). An example of the later analysis is provided in an RNA-Seq workflow on the Bioconductor website:
http://www.bioconductor.org/help/workflows/rnaseqGene/.
<a id="3.5"></a>
### 3.5 Likelihood ratio test
*DESeq2* offers two kinds of hypothesis tests: the Wald test, where we use the estimated standard error of
a log2 fold change to test if it is equal to zero, and the likelihood ratio test (LRT). The LRT examines two
models for the counts, a *full* model with a certain number of terms and a *reduced* model, in which some of
the terms of the *full* model are removed. The test determines if the increased likelihood of the data using the
extra terms in the *full* model is more than expected if those extra terms are truly zero.

The LRT is therefore useful for testing multiple terms at once, for example testing 3 or more levels of a factor
at once, or all interactions between two variables. The LRT for count data is conceptually similar to an analysis
of variance (ANOVA) calculation in linear regression, except that in the case of the Negative Binomial GLM, we use an analysis of deviance (ANODEV), where the *deviance* captures the difference in likelihood between
a full and a reduced model.

The likelihood ratio test can be specified using the test argument to <font face='courier'>DESeq</font>, which substitutes <font face='courier'>nbinomWaldTest</font>
with <font face='courier'>nbinomLRT</font>. In this case, the user needs to provide a reduced formula, e.g. one in which a number of
terms from <font face='courier'>design(dds)</font> are removed. The degrees of freedom for the test is obtained from the difference
between the number of parameters in the two models.
<a id="3.6"></a>
### 3.6 Approach to count outliers
RNA-Seq data sometimes contain isolated instances of very large counts that are apparently unrelated to the
experimental or study design, and which may be considered outliers. There are many reasons why outliers
can arise, including rare technical or experimental artifacts, read mapping problems in the case of genetically
differing samples, and genuine, but rare biological events. In many cases, users appear primarily interested in
genes that show a consistent behavior, and this is the reason why by default, genes that are affected by such
outliers are set aside by *DESeq2*, or if there are sufficient samples, outlier counts are replaced for model fitting.
These two behaviors are described below.

The <font face='courier'>DESeq</font> function (and <font face='courier'>nbinomWaldTest/nbinomLRT</font> functions) calculates, for every gene and for every
sample, a diagnostic test for outliers called *Cook’s distance*. Cook’s distance is a measure of how much a
single sample is influencing the fitted coefficients for a gene, and a large value of Cook’s distance is intended to
indicate an outlier count. The Cook’s distances are stored as a matrix available in <font face='courier'>assays(dds)[["cooks"]]</font>.
The <font face='courier'>results</font> function automatically flags genes which contain a Cook’s distance above a cutoff for samples
which have 3 or more replicates. The $p$ values and adjusted $p$ values for these genes are set to <font face='courier'>NA</font>. At least
3 replicates are required for flagging, as it is difficult to judge which sample might be an outlier with only 2
replicates.

With many degrees of freedom – i. e., many more samples than number of parameters to be estimated – it
is undesirable to remove entire genes from the analysis just because their data include a single count outlier.
When there are 7 or more replicates for a given sample, the <font face='courier'>DESeq</font> function will automatically replace counts
with large Cook’s distance with the trimmed mean over all samples, scaled up by the size factor or normalization
factor for that sample. This approach is conservative, it will not lead to false positives, as it replaces the outlier
value with the value predicted by the null hypothesis.

The default Cook’s distance cutoff for the two behaviors described above depends on the sample size and
number of parameters to be estimated. The default is to use the 99% quantile of the $F (p, m − p)$ distribution
(with $p$ the number of parameters including the intercept and $m$ number of samples). The default for gene
flagging can be modified using the cooksCutoff argument to the results function. The gene flagging
functionality can be disabled by setting <font face='courier'>cooksCutoff</font> to <font face='courier'>FALSE</font> or <font face='courier'>Inf</font>. The automatic outlier replacement
performed by <font face='courier'>DESeq</font> can be disabled by setting the <font face='courier'>minReplicatesForReplace</font> argument to <font face='courier'>Inf</font>.

<font face='courier'>DESeq</font> automatically replaces outliers if there are sufficient replicates and a row contains a count with very
high Cook’s distance. <font face='courier'>DESeq</font> preserves the original counts in <font face='courier'>counts(dds)</font> saving the replacement counts as
a matrix named <font face='courier'>replaceCounts</font> in <font face='courier'>assays(dds)</font>.

Figure 9: Dispersion plot. The dispersion estimate plot shows the gene-wise estimates (black), the fitted
values (red), and the final maximum a posteriori estimates used in testing (blue).
<a id="3.7"></a>
### 3.7 Dispersion plot and fitting alternatives
Plotting the dispersion estimates is a useful diagnostic. The dispersion plot in Figure 9 is typical, with the
final estimates shrunk from the gene-wise estimates towards the fitted estimates. Some gene-wise estimates
are flagged as outliers and not shrunk towards the fitted value, (this outlier detection is described in the man
page for <font face='courier'>estimateDispersionsMAP</font>). The amount of shrinkage can be more or less than seen here, depending
on the sample size, the number of coefficients, the row mean and the variability of the gene-wise estimates.


In [ ]:
plotDispEsts(dds)

<a id="3.7.1"></a>
#### 3.7.1 Local or mean dispersion fit
A local smoothed dispersion fit is automatically substitited in the case that the parametric curve doesn’t fit the
observed dispersion mean relationship. This can be prespecified by providing the argument <font face='courier'>fitType="local"</font>
to either <font face='courier'>DESeq</font> or <font face='courier'>estimateDispersions</font>. Additionally, using the mean of gene-wise disperion estimates as
the fitted value can be specified by providing the argument <font face='courier'>fitType="mean"</font>.
<a id="3.7.2"></a>
#### 3.7.2 Supply a custom dispersion fit
Any fitted values can be provided during dispersion estimation, using the lower-level functions described in the
manual page for <font face='courier'>estimateDispersionsGeneEst</font>. In the code chunk below, we store the gene-wise estimates which were already calculated and saved in the metadata column <font face='courier'>dispGeneEst</font>. Then we calculate the median
value of the dispersion estimates above a threshold, and save these values as the fitted dispersions, using the
replacement function for <font face='courier'>dispersionFunction</font>. In the last line, the function <font face='courier'>estimateDispersionsMAP</font>,
uses the fitted dispersions to generate maximum a posteriori (MAP) estimates of dispersion.

In [ ]:
ddsCustom <- dds
useForMedian <- mcols(ddsCustom)$dispGeneEst > 1e-7
medianDisp <- median(mcols(ddsCustom)$dispGeneEst[useForMedian],na.rm=TRUE)
dispersionFunction(ddsCustom) <- function(mu) medianDisp
ddsCustom <- estimateDispersionsMAP(ddsCustom)

<a id="3.8"></a>
### 3.8 Independent filtering of results
The <font face='courier'>results</font> function of the *DESeq2* package performs independent filtering by default using the mean of
normalized counts as a filter statistic. A threshold on the filter statistic is found which optimizes the number of
adjusted $p$ values lower than a significance level alpha (we use the standard variable name for significance level,
though it is unrelated to the dispersion parameter $\alpha$). The theory behind independent filtering is discussed in
greater detail in Section 4.7. The adjusted $p$ values for the genes which do not pass the filter threshold are set
to NA.

The independent filtering is performed using the <font face='courier'>filtered_p</font> function of the [**genefilter**](http://bioconductor.org/packages/release/bioc/html/genefilter.html) package, and all of
the arguments of <font face='courier'>filtered_p</font> can be passed to the <font face='courier'>results</font> function. The filter threshold value and the
number of rejections at each quantile of the filter statistic are available as attributes of the object returned
by <font face='courier'>results</font>. For example, we can visualize the optimization by plotting the <font face='courier'>filterNumRej</font> attribute of the
results object, as seen in Figure 10. Note that if the maximum number of rejections is very small such that
the line of rejections over filter threshold appears noisy, the expected false discovery rate might not be held
exactly for this small set.

In [ ]:
attr(res,"filterThreshold")
plot(attr(res,"filterNumRej"),type="b",
     ylab="number of rejections")

Independent filtering can be turned off by setting <font face='courier'>independentFiltering</font> to <font face='courier'>FALSE</font>.

In [ ]:
resNoFilt <- results(dds, independentFiltering=FALSE)
addmargins(table(filtering=(res$padj < .1), noFiltering=(resNoFilt$padj < .1)))

Figure 10: Independent filtering. The results function maximizes the number of rejections (adjusted p
value less than a significance level), over theta, the quantiles of a filtering statistic (in this case, the mean of
normalized counts).
<a id="3.9"></a>
### 3.9 Tests of log2 fold change above or below a threshold
It is also possible to provide thresholds for constructing Wald tests of significance. Two arguments to the
<font face='courier'>results</font> function allow for threshold-based Wald tests: <font face='courier'>lfcThreshold</font>, which takes a numeric of a non-
negative threshold value, and <font face='courier'>altHypothesis</font>, which specifies the kind of test. Note that the *alternative
hypothesis* is specified by the user, i.e. those genes which the user is interested in finding, and the test provides
$p$ values for the null hypothesis, the complement of the set defined by the alternative. The <font face='courier'>altHypothesis</font>
argument can take one of the following four values, where $\beta$ is the log2 fold change specified by the <font face='courier'>name</font>
argument:
* greaterAbs - |β| > lfcThreshold - tests are two-tailed
* lessAbs - |β| < lfcThreshold - p values are the maximum of the upper and lower tests
* greater - β > lfcThreshold
* less - β < −lfcThreshold

The test <font face='courier'>altHypothesis="lessAbs"</font> requires that the user have run <font face='courier'>DESeq</font> with the argument betaPrior=FALSE.
To understand the reason for this requirement, consider that during hypothesis testing, the null hypothesis is
favored unless the data provide strong evidence to reject the null. For this test, including a zero-centered prior
on log fold change would favor the alternative hypothesis, shrinking log fold changes toward zero. Removing
the prior on log fold changes for tests of small log fold change allows for detection of only those genes where
the data alone provides evidence against the null.

Figure 11: MA-plots of tests of log2 fold change with respect to a threshold value. Going left to right
across rows, the tests are for altHypothesis = "greaterAbs", "lessAbs", "greater", and "less".



The four possible values of altHypothesis are demonstrated in the following code and visually by MA-plots in
Figure 11. First we run DESeq and specify betaPrior=FALSE in order to demonstrate altHypothesis="lessAbs"

In [ ]:
ddsNoPrior <- DESeq(dds, betaPrior=FALSE)

In order to produce results tables for the following tests, the same arguments (except <font face='courier'>ylim</font>) would be provided
to the <font face='courier'>results</font> function.

In [ ]:
par(mfrow=c(2,2),mar=c(2,2,1,1))
yl <- c(-2.5,2.5)
resGA <- results(dds, lfcThreshold=.5, altHypothesis="greaterAbs")
resLA <- results(ddsNoPrior, lfcThreshold=.5, altHypothesis="lessAbs")
resG <- results(dds, lfcThreshold=.5, altHypothesis="greater")
resL <- results(dds, lfcThreshold=.5, altHypothesis="less")
plotMA(resGA, ylim=yl)
abline(h=c(-.5,.5),col="dodgerblue",lwd=2)
plotMA(resLA, ylim=yl)
abline(h=c(-.5,.5),col="dodgerblue",lwd=2)
plotMA(resG, ylim=yl)
abline(h=.5,col="dodgerblue",lwd=2)
plotMA(resL, ylim=yl)
abline(h=-.5,col="dodgerblue",lwd=2)

<a id="3.10"></a>
### 3.10 Access to all calculated values
All row-wise calculated values (intermediate dispersion calculations, coefficients, standard errors, etc.) are
stored in the *DESeqDataSet* object, e.g. <font face='courier'>dds</font> in this vignette. These values are accessible by calling <font face='courier'>mcols</font> on <font face='courier'>dds</font>. Descriptions of the columns are accessible by two calls to <font face='courier'>mcols</font>.

In [ ]:
mcols(dds,use.names=TRUE)[1:4,1:4]

In [ ]:
# here using substr() only for display purposes
substr(names(mcols(dds)),1,10)

In [ ]:
mcols(mcols(dds), use.names=TRUE)[1:4,]

The mean values $\mu_{ij} = s_j q_{ij}$ and the Cook’s distances for each gene and sample are stored as matrices in the
assays slot:

In [ ]:
head(assays(dds)[["mu"]])

In [ ]:
head(assays(dds)[["cooks"]])

The dispersions $\alpha_i$ can be accessed with the <font face='courier'>dispersions</font> function.

In [ ]:
head(dispersions(dds))

In [ ]:
# which is the same as
head(mcols(dds)$dispersion)

The size factors $s_j$ are accessible via <font face='courier'>sizeFactors</font>:

In [ ]:
sizeFactors(dds)

For advanced users, we also include a convenience function <font face='courier'>coef</font> for extracting the matrix of coefficients $[\beta_{ir} ]$
for all genes $i$ and parameters $r$, as in the formula in Section 4.1. This function can also return a matrix of
standard errors, see <font face='courier'>?coef</font>. The columns of this matrix correspond to the effects returned by <font face='courier'>resultsNames</font>.
Note that the <font face='courier'>results</font> function is best for building results tables with $p$ values and adjusted $p$ values.

In [ ]:
head(coef(dds))

The beta prior variance $\sigma_r^2$ is stored as an attribute of the *DESeqDataSet*:

In [ ]:
attr(dds, "betaPriorVar")

The dispersion prior variance $\sigma_d^2$ is stored as an attribute of the <font face='courier'>dispersion</font> function:

In [ ]:
dispersionFunction(dds)

In [ ]:
attr(dispersionFunction(dds), "dispPriorVar")

<a id="3.11"></a>
### 3.11 Sample-/gene-dependent normalization factors
In some experiments, there might be gene-dependent dependencies which vary across samples. For instance,
GC-content bias or length bias might vary across samples coming from different labs or processed at different
times. We use the terms “normalization factors” for a gene x sample matrix, and “size factors” for a single
number per sample. Incorporating normalization factors, the mean parameter $\mu_{ij}$ from Section 4.1 becomes:
$$\mu_{ij}=NF_{ij}q_{ij}$$
with normalization factor matrix $NF$ having the same dimensions as the counts matrix $K$. This matrix can
be incorporated as shown below. We recommend providing a matrix with row-wise geometric means of 1, so
that the mean of normalized counts for a gene is close to the mean of the unnormalized counts. This can be
accomplished by dividing out the current row geometric means.

```r
normFactors <- normFactors / exp(rowMeans(log(normFactors)))
normalizationFactors(dds) <- normFactors
```

These steps then replace <font face='courier'>estimateSizeFactors</font> in the steps described in Section 3.1. Normalization factors,
if present, will always be used in the place of size factors.

The methods provided by the [**cqn**](http://bioconductor.org/packages/release/bioc/html/cqn.html) or [**EDASeq**](http://bioconductor.org/packages/release/bioc/html/EDASeq.html) packages can help correct for GC or length biases. They both
describe in their vignettes how to create matrices which can be used by *DESeq2* . From the formula above, we
see that normalization factors should be on the scale of the counts, like size factors, and unlike offsets which
are typically on the scale of the predictors (i.e. the logarithmic scale for the negative binomial GLM). At the
time of writing, the transformation from the matrices provided by these packages should be:

```r
cqnOffset <- cqnObject$glm.offset
cqnNormFactors <- exp(cqnOffset)
EDASeqNormFactors <- exp(-1 * EDASeqOffset)
```

<a id="3.12"></a>
### 3.12 Model matrix not full rank
While most experimental designs run easily using design formula, some design formulas can cause problems
and result in the <font face='courier'>DESeq</font> function returning an error with the text: “the model matrix is not full rank, so the
model cannot be fit as specified.” There are two reasons for this problem, either one or more columns in the
model matrix are linear combinations of other columns, or there are levels of factors or combinations of levels
of multiple factors which are missing samples. We address these problems below and discuss possible solutions:
<a id="3.12.1"></a>
#### 3.12.1 Linear combinations
The simplest case of the linear combination, or linear dependency problem, is if two variables contain exactly
the same information, such as in the following sample table. The software cannot fit an effect for <font face='courier'>batch</font>
and <font face='courier'>condition</font>, because they produce identical columns in the model matrix. This is also referred to as
“confounding”. A unique solution of coefficients (the $\beta_i$ in the following Section 4.1) is not possible.
```r
##    batch condition 
## 1      1         A
## 2      1         A
## 3      2         B
## 4      2         B
```
Another situation which will cause problems is when the variables are not identical, but one variable can be
formed by the combination of other factor levels. In the following example, the effect of batch 2 vs 1 cannot
be fit because it is identical to a column in the model matrix which represents the condition C vs A effect.
```r
##    batch condition 
## 1      1         A
## 2      1         A
## 3      1         B
## 4      1         B
## 5      2         C
## 6      2         C
```
In both of these cases, the batch effect cannot be fit and must be removed from the model formula. The
options are either to assume there is no batch effect (which we know is highly unlikely given the literature on
batch effects in sequencing datasets) or to repeat the experiment and properly balance the conditions across
batches, for example:

```r
##    batch condition 
## 1      1         A
## 2      1         B
## 3      1         C
## 4      2         A
## 5      2         B
## 6      2         C
```
Finally, there is the case of an experiment with grouped individuals, where a group-specific effect of some
treatment is sought. This design requires a bit of refactoring in order to fit the model terms. Here we use
a trick from the [**edgeR**](http://bioconductor.org/packages/release/bioc/html/edgeR.html) user guide, from the section “Comparisons Both Between and Within Subjects”. An
example of such an experimental design is:

In [ ]:
(coldata <- data.frame(grp=factor(rep(c("X","Y"),each=4)),
                       ind=factor(rep(1:4,each=2)),
                       cnd=factor(rep(c("A","B"),4))))

If we try to analyze with a formula such as, <font face='courier'>∼ ind + grp*cnd</font>, we will obtain an error, because the effect for
group is a linear combination of the individuals.
However, the following steps allow for an analysis of group-specific condition effects, while controlling for
differences in individual. For object construction, use a dummy design, such as ∼ 1. Then add a column
<font face='courier'>ind.n</font> which distinguishes the individuals “nested” within a group. Here, we add this column to coldata, but
in practice you would add this column to <font face='courier'>dds</font>.

In [ ]:
coldata$ind.n <- factor(rep(rep(1:2,each=2),2))
coldata

Now we can reassign our *DESeqDataSet* a design of <font face='courier'>∼ grp + grp:ind.n + grp:cnd</font>, before we call <font face='courier'>DESeq</font>.
This new design will result in the following model matrix:

In [ ]:
model.matrix(~ grp + grp:ind.n + grp:cnd, coldata)

where the terms <font face='courier'>grpX.cndB</font> and <font face='courier'>grpY.cndB</font> give the group-specific condition effects.

<a id="3.12.2"></a>
#### 3.12.2 Levels without samples
The base R function for creating model matrices will produce a column of zeros if a level is missing from a
factor or a combination of levels is missing from an interaction of factors. The solution to the first case is to
call <font face='courier'>droplevels</font> on the column, which will remove levels without samples. This was shown in the beginning
of this vignette.

The second case is also solvable, by manually editing the model matrix, and then providing this to <font face='courier'>DESeq</font>.
Here we construct an example dataset to illustrate:

In [ ]:
group <- factor(rep(1:3,each=6))
condition <- factor(rep(rep(c("A","B","C"),each=2),3))
(d <- data.frame(group, condition)[-c(17,18),])

Note that if we try to estimate all interaction terms, we introduce a column with all zeros, as there are no
condition C samples for group 3. (Here, <font face='courier'>unname</font> is used to display the matrix concisely.)

In [ ]:
m1 <- model.matrix(~ condition*group, d)
colnames(m1)

In [ ]:
unname(m1)

We can remove this column like so:

In [ ]:
m1 <- m1[,-9]
unname(m1)

Now this matrix <font face='courier'>m1</font> can be provided to the <font face='courier'>full</font> argument of <font face='courier'>DESeq</font>. For a likelihood ratio test of interactions,
a model matrix using a reduced design such as <font face='courier'>∼ condition + group</font> can be given to the reduced argument.
Wald tests can also be generated instead of the likelihood ratio test, but for user-supplied model matrices, the
argument <font face='courier'>betaPrior</font> must be set to <font face='courier'>FALSE</font>.
<a id="4"></a>
## 4 Theory behind DESeq2
<a id="4.1"></a>
### 4.1 The DESeq2 model
The *DESeq2* model and all the steps taken in the software are described in detail in our pre-print [1], and we
include the formula and descriptions in this section as well. The differential expression analysis in *DESeq2* uses
a generalized linear model of the form:

\begin{align*} K_{ij} &\sim \mathrm{NB}(\mu_{ij},\,\alpha_i)\\ \mu_{ij} &=s_jq_{ij} \\ \log_2(q_{ij})&=x_j\beta_i  \end{align*}
    
    where counts K ij for gene i, sample j are modeled using a negative binomial distribution with fitted mean μ ij
and a gene-specific dispersion parameter $\alpha_i$. The fitted mean is composed of a sample-specific size factor $s_j$
and a parameter $q_{ij}$ proportional to the expected true concentration of fragments for sample $j$. The coefficients
$\beta_i$ give the log2 fold changes for gene $i$ for each column of the model matrix X.
By default these log2 fold changes are the maximum a posteriori estimates after incorporating a zero-centered
Normal prior – in the software referrred to as a $\beta$-prior – hence *DESeq2* provides “moderated” log2 fold change
estimates. Dispersions are estimated using expected mean values from the maximum likelihood estimate of
log2 fold changes, and optimizing the Cox-Reid adjusted profile likelihood, as first implemented for RNA-Seq
data in [**edgeR**](http://bioconductor.org/packages/release/bioc/html/edgeR.html) [7, 8]. The steps performed by the DESeq function are documented in its manual page; briefly,
they are:
1. estimation of size factors $s_j$ by $\mathtt{estimateSizeFactors}$
2. estimation of dispersion $\alpha_i$ by $\mathtt{estimateDispersions}$
3. negative binomial GLM fitting for $\beta_i$ and Wald statistics by $\mathtt{nbinomWaldTest}$

For access to all the values calculated during these steps, see Section 3.10
<a id="4.2"></a>
### 4.2  Changes compared to the DESeq package
The main changes in the package *DESeq2* , compared to the (older) version [**DESeq**](http://bioconductor.org/packages/release/bioc/html/DESeq.html), are as follows:
* *SummarizedExperiment* is used as the superclass for storage of input data, intermediate calculations and
results.
* Maximum a posteriori estimation of GLM coefficients incorporating a zero-centered Normal prior with
variance estimated from data (equivalent to Tikhonov/ridge regularization). This adjustment has little
effect on genes with high counts, yet it helps to moderate the otherwise large variance in log2 fold change
estimates for genes with low counts or highly variable counts.
* Maximum a posteriori estimation of dispersion replaces the sharingMode options fit-only or maximum
of the previous version of the package. This is similar to the dispersion estimation methods of DSS [9].
* All estimation and inference is based on the generalized linear model, which includes the two condition
case (previously the exact test was used).
* The Wald test for significance of GLM coefficients is provided as the default inference method, with the
likelihood ratio test of the previous version still available.

* It is possible to provide a matrix of sample-/gene-dependent normalization factors (Section 3.11).
* Automatic independent filtering on the mean of normalized counts (Section 4.7).
* Automatic outlier detection and handling (Section 4.4).
<a id="4.3"></a>
### 4.3  Methods changes since the 2014 DESeq2 paper
* For the calculation of the beta prior variance, instead of simply matching the empirical quantile to the
quantile of a Normal distribution, we use the weighted quantile function of the [**Hmisc**](http://cran.fhcrc.org/web/packages/Hmisc/index.html) package. The
weighting is described in the man page for <font face='courier'>nbinomWaldTest</font>. The weights used are the inverse of the
expected variance of log counts (as used in the diagonals of the matrix W in the GLM). The effect is
that the estimated prior variance is robust against noisy estimates of log fold change from genes with
very small counts.
<a id="4.4"></a>
### 4.4 Count outlier detection
*DESeq2* relies on the negative binomial distribution to make estimates and perform statistical inference on
differences. While the negative binomial is versatile in having a mean and dispersion parameter, extreme
counts in individual samples might not fit well to the negative binomial. For this reason, we perform automatic
detection of count outliers. We use Cook’s distance, which is a measure of how much the fitted coefficients
would change if an individual sample were removed [10]. For more on the implementation of Cook’s distance
see Section 3.6 and the manual page for the results function. Below we plot the maximum value of Cook’s
distance for each row over the rank of the test statistic to justify its use as a filtering criterion.

In [ ]:
W <- res$stat
maxCooks <- apply(assays(dds)[["cooks"]],1,max)
idx <- !is.na(W)
plot(rank(W[idx]), maxCooks[idx], xlab="rank of Wald statistic",
     ylab="maximum Cook's distance per gene",
     ylim=c(0,5), cex=.4, col=rgb(0,0,0,.3))
m <- ncol(dds)
p <- 3
abline(h=qf(.99, p, m - p))

<a id="4.5"></a>
### 4.5 Contrasts
Contrasts can be calculated for a *DESeqDataSet* object for which the GLM coefficients have already been fit
using the Wald test steps (<font face='courier'>DESeq</font> with <font face='courier'>test="Wald"</font> or using <font face='courier'>nbinomWaldTest</font>). The vector of coefficients
$\beta$ is left multiplied by the contrast vector $c$ to form the numerator of the test statistic. The denominator is
formed by multiplying the covariance matrix $\Sigma$ for the coefficients on either side by the contrast vector $c$. The
square root of this product is an estimate of the standard error for the contrast. The contrast statistic is then
compared to a normal distribution as are the Wald statistics for the *DESeq2* package.
$$ W = \frac{c^t\beta}{\sqrt{c^t\sum c}}$$

Figure 12: Cook’s distance. Plot of the maximum Cook’s distance per gene over the rank of the Wald
statistics for the condition. The two regions with small Cook’s distances are genes with a single count in one
sample. The horizontal line is the default cutoff used for 7 samples and 3 estimated parameters.
<a id="4.6"></a>
### 4.6 Expanded model matrices
As mentioned in Section 3.2, *DESeq2* uses “expanded model matrices” with the log2 fold change prior, in order
to produce log2 fold change estimates and test results which are independent of the choice of reference level.
These model matrices differ from the standard model matrices, in that they have an indicator column (and
therefore a coefficient) for each level of factors in the design formula in addition to an intercept. Expanded
model matrices are not used without the log2 fold change prior or in the case of designs with 2 level factors
and an interaction term.

These matrices are therefore not full rank, but a coefficient vector $\bea_i$ can still be found due to the zero-centered
prior on non-intercept coefficients. The prior variance for the log2 fold changes is calculated by first generating
maximum likelihood estimates for a standard model matrix. The prior variance for each level of a factor is
then set as the average of the mean squared maximum likelihood estimates for each level and every possible
contrast, such that that this prior value will be reference-level-independent. The <font face='courier'>contrast</font> argument of the <font face='courier'>results</font> function is again used in order to generate comparisons of interest.
<a id="4.7"></a>
### 4.7 Independent filtering and multiple testing
<a id="4.7.1"></a>
#### 4.7.1  Filtering criteria
The goal of independent filtering is to filter out those tests from the procedure that have no, or little chance
of showing significant evidence, without even looking at their test statistic. Typically, this results in increased
detection power at the same experiment-wide type I error. Here, we measure experiment-wide type I error in
terms of the false discovery rate.
A good choice for a filtering criterion is one that
1. is statistically independent from the test statistic under the null hypothesis,
2. is correlated with the test statistic under the alternative, and
3. does not notably change the dependence structure – if there is any– between the tests that pass the filter,
compared to the dependence structure between the tests before filtering.

The benefit from filtering relies on property 2, and we will explore it further in Section 4.7.2. Its statistical
validity relies on property 1 – which is simple to formally prove for many combinations of filter criteria with
test statistics– and 3, which is less easy to theoretically imply from first principles, but rarely a problem in
practice. We refer to [11] for further discussion of this topic.

A simple filtering criterion readily available in the results object is the mean of normalized counts irrespective of
biological condition (Figure 13). Genes with very low counts are not likely to see significant differences typically
due to high dispersion. For example, we can plot the $−\log_{10} p$ values from all genes over the normalized mean
counts.

In [ ]:
plot(res$baseMean+1, -log10(res$pvalue),
     log="x", xlab="mean of normalized counts",
     ylab=expression(-log[10](pvalue)),
     ylim=c(0,30),
     cex=.4, col=rgb(0,0,0,.3))

<a id="4.7.1"></a>
#### 4.7.2 Why does it work?
Consider the $p$ value histogram in Figure 14. It shows how the filtering ameliorates the multiple testing problem
– and thus the severity of a multiple testing adjustment – by removing a background set of hypotheses whose
$p$ values are distributed more or less uniformly in [0, 1].

In [ ]:
use <- res$baseMean > attr(res,"filterThreshold")
table(use)

In [ ]:
h1 <- hist(res$pvalue[!use], breaks=0:50/50, plot=FALSE)
h2 <- hist(res$pvalue[use], breaks=0:50/50, plot=FALSE)
colori <- c(`do not pass`="khaki", `pass`="powderblue")

In [ ]:
barplot(height = rbind(h1$counts, h2$counts), beside = FALSE,
col = colori, space = 0, main = "", ylab="frequency")
text(x = c(0, length(h1$counts)), y = 0, label = paste(c(0,1)),
adj = c(0.5,1.7), xpd=NA)
legend("topright", fill=rev(colori), legend=rev(names(colori)))

Figure 13: Mean counts as a filter statistic. The mean of normalized counts provides an independent
statistic for filtering the tests. It is independent because the information about the variables in the design
formula is not used. By filtering out genes which fall on the left side of the plot, the majority of the low p
values are kept.

<a id="4.7.3"></a>
#### 4.7.3 Diagnostic plots for multiple testing
The Benjamini-Hochberg multiple testing adjustment procedure [12] has a simple graphical illustration, which
we produce in the following code chunk. Its result is shown in the left panel of Figure 15.

In [ ]:
resFilt <- res[use & !is.na(res$pvalue),]
orderInPlot <- order(resFilt$pvalue)
showInPlot <- (resFilt$pvalue[orderInPlot] <= 0.08)
alpha <- 0.1

In [ ]:
plot(seq(along=which(showInPlot)), resFilt$pvalue[orderInPlot][showInPlot],
     pch=".", xlab = expression(rank(p[i])), ylab=expression(p[i]))
abline(a=0, b=alpha/length(resFilt$pvalue), col="red3", lwd=2)

Schweder and Spjøtvoll [13] suggested a diagnostic plot of the observed $p$-values which permits estimation
of the fraction of true null hypotheses. For a series of hypothesis tests $H_1 , \dots , H_m$ with $p$-values $p_i$, they
suggested plotting

$$ (1-p_i,\,N(p_i))\,\,\mathrm{for}\,\,i\in 1,\dots,m, $$

Figure 14: Histogram of p values for all tests. The area shaded in blue indicates the subset of those that
pass the filtering, the area in khaki those that do not pass.


where $N (p)$ is the number of $p$-values greater than $p$. An application of this diagnostic plot to 
<font face='courier'>resFilt\$pvalue </font> is shown in the right panel of Figure 15. When all null hypotheses are true, the $p$-values are each uniformly
distributed in [0, 1], Consequently, the cumulative distribution function of $(p_1 , \dots , p_m )$ is expected to be close
to the line $F (t) = t$. By symmetry, the same applies to $(1 − p_1 , \dots , 1 − p_m )$. When (without loss of generality)
the first $m_0$ null hypotheses are true and the other $m − m_0$ are false, the cumulative distribution function of
$(1 − p_1 , \dots, 1 − p_{m_0} )$ is again expected to be close to the line $F_0 (t) = t$. The cumulative distribution function
of $(1 − p_{m_0 +1} , \dots , 1 − p_m )$, on the other hand, is expected to be close to a function $F_1 (t)$ which stays below
$F_0$ but shows a steep increase towards 1 as $t$ approaches 1. In practice, we do not know which of the null
hypotheses are true, so we can only observe a mixture whose cumulative distribution function is expected to
be close to

$$F(t)=\frac{m_0}{m}F_0(t)+ \frac{m-m_0}{m}F_1(t)$$

Such a situation is shown in the right panel of Figure 15. If $F_1 (t)/F_0 (t)$ is small for small $t$, then the mixture
fraction $\frac{m_0}{m}$ can be estimated by fitting a line to the left-hand portion of the plot, and then noting its height
on the right. Such a fit is shown by the red line in the right panel of Figure 15.

In [ ]:
plot(1-resFilt$pvalue[orderInPlot],
    (length(resFilt$pvalue)-1):0, pch=".",
     xlab=expression(1-p[i]), ylab=expression(N(p[i])))
abline(a=0, slope, col="red3", lwd=2)

<a id="5"></a>
## 5 Frequently asked questions
<a id="5.1"></a>
### 5.1 How can I get support for DESeq2?
We welcome questions about our software, and want to ensure that we eliminate issues if and when they
appear. We have a few requests to optimize the process:
* all questions should take place on the Bioconductor support site: https://support.bioconductor.
org, which serves as a repository of questions and answers. This helps to save the developers’ time in
responding to similar questions. Make sure to tag your post with “deseq2”. It is often very helpful in
addition to describe the aim of your experiment.
* before posting, first search the Bioconductor support site mentioned above for past threads which might
have answered your question
* if you have a question about the behavior of a function, read the sections of the manual page for this
function by typing a question mark and the function name, e.g. <font face='courier'>?results</font>. We spend a lot of time
documenting individual functions and the exact steps that the software is performing.
* include all of your R code, especially the creation of the *DESeqDataSet* and the design formula. Include
complete warning or error messages, and conclude your message with the full output of <font face='courier'>sessionInfo()</font>.
* if possible, include the output of <font face='courier'>as.data.frame(colData(dds))</font>, so that we can have a sense of the
experimental setup. If this contains confidential information, you can replace the levels of those factors
using <font face='courier'>levels()</font>.
<a id="5.2"></a>
### 5.2 Why are some $p$ values set to NA?
See the details in Section 1.4.3.
<a id="5.3"></a>
### 5.3 How can I get unfiltered <font face='courier'>DESeq</font> results?
Users can obtain unfiltered GLM results, i.e. without outlier removal or independent filtering with the following
call:

In [ ]:
dds <- DESeq(dds, minReplicatesForReplace=Inf)
res <- results(dds, cooksCutoff=FALSE, independentFiltering=FALSE)

In this case, the only p values set to NA are those from genes with all counts equal to zero.
<a id="5.4"></a>
### 5.4 How do I use the variance stabilized or rlog transformed data for differential testing?
The variance stabilizing and rlog transformations are provided for applications other than differential testing,
for example clustering of samples or other machine learning applications. For differential testing we recommend
the DESeq function applied to raw counts as outlined in Section 1.3.
<a id="5.5"></a>
### 5.5 Can I use DESeq2 to analyze paired samples?
Yes, you should use a multi-factor design which includes the sample information as a term in the design formula.
This will account for differences between the samples while estimating the effect due to the condition. The
condition of interest should go at the end of the design formula. See Section 1.5.
<a id="5.6"></a>
### 5.6 Can I run DESeq2 to contrast the levels of 100 groups?
*DESeq2* will work with any kind of design specified using the R formula. We enourage users to consider
exploratory data analysis such as principal components analysis as described in Section 2.2.3, rather than
performing statistical testing of all combinations of dozens of groups.
As a speed concern with fitting very large models, note that each additional level of a factor in the design
formula adds another parameter to the GLM which is fit by *DESeq2*. Users might consider first removing genes
with very few reads, e.g. genes with row sum of 1, as this will speed up the fitting procedure.
<a id="5.7"></a>
### 5.7 Can I use DESeq2 to analyze a dataset without replicates?
If a *DESeqDataSet* is provided with an experimental design without replicates, a message is printed, that the
samples are treated as replicates for estimation of dispersion. More details can be found in the manual page
for <font face='courier'>?DESeq</font>.
<a id="5.8"></a>
### 5.8 How can I include a continuous covariate in the design formula?
Continuous covariates can be included in the design formula in the same manner as factorial covariates.
Continuous covariates might make sense in certain experiments, where a constant fold change might be
expected for each unit of the covariate. However, in many cases, more meaningful results can be obtained by
cutting continuous covariates into a factor defined over a small number of bins (e.g. 3-5). In this way, the
average effect of each group is controlled for, regardless of the trend over the continuous covariates. In R,
numeric vectors can be converted into factors using the function cut.
<a id="5.9"></a>
### 5.9 What are the exact steps performed by <font face='courier'>DESeq()</font>?
See the manual page for <font face='courier'>DESeq</font>, which links to the subfunctions which are called in order, where complete
details are listed.
<a id="6"></a>
## 6 Acknowledgments
We have benefited in the development of *DESeq2* from the help and feedback of many individuals, including
but not limited to: The Bionconductor Core Team, Alejandro Reyes, Andrzej Oles, Aleksandra Pekowska, Felix
Klein, Vince Carey, Devon Ryan, Steve Lianoglou, Jessica Larson, Christina Chaivorapol, Pan Du, Richard
Bourgon, Willem Talloen, Elin Videvall, Hanneke van Deutekom, Todd Burwell, Jesse Rowley, Igor Dolgalev,
Stephen Turner, Ryan C Thompson, Tyr Wiesner-Hanks, Konrad Rudolph, David Robinson.
<a id="ref"></a>
## References
[1] Michael I. Love, Wolfgang Huber, and Simon Anders. Moderated estimation of fold change and dispersion
for RNA-seq data with DESeq2. Genome Biology, 15:550, 2014. URL: http://dx.doi.org/10.1186/s13059-014-0550-8.

[2] Simon Anders, Paul Theodor Pyl, and Wolfgang Huber. HTSeq – A Python framework to work
with high-throughput sequencing data. Bioinformatics, 2014. URL: http://dx.doi.org/10.1093/bioinformatics/btu638.

[3] A. N. Brooks, L. Yang, M. O. Duff, K. D. Hansen, J. W. Park, S. Dudoit, S. E. Brenner, and B. R.
Graveley. Conservation of an RNA regulatory map between Drosophila and mammals. Genome Research,
pages 193–202, 2011. URL: http://genome.cshlp.org/cgi/doi/10.1101/gr.108662.110, doi:
10.1101/gr.108662.110.

[4] Robert Tibshirani. Estimating transformations for regression via additivity and variance stabilization.
Journal of the American Statistical Association, 83:394–405, 1988.

[5] Wolfgang Huber, Anja von Heydebreck, Holger Sültmann, Annemarie Poustka, and Martin Vingron. Pa-
rameter estimation for the calibration and variance stabilization of microarray data. Statistical Applications
in Genetics and Molecular Biology, 2(1):Article 3, 2003.

[6] Simon Anders and Wolfgang Huber. Differential expression analysis for sequence count data. Genome
Biology, 11:R106, 2010. URL: http://genomebiology.com/2010/11/10/R106.

[7] D. R. Cox and N. Reid. Parameter orthogonality and approximate conditional inference. Journal of the
Royal Statistical Society, Series B, 49(1):1–39, 1987. URL: http://www.jstor.org/stable/2345476.

[8] Davis J McCarthy, Yunshun Chen, and Gordon K Smyth. Differential expression analysis of multifactor
RNA-Seq experiments with respect to biological variation. Nucleic Acids Research, 40:4288–4297, January
2012 http://www.ncbi.nlm.nih.gov/pubmed/22287627, doi:10.1093/nar/gks042.

[9] Hao Wu, Chi Wang, and Zhijin Wu. A new shrinkage estimator for dispersion improves differential
expression detection in RNA-seq data. Biostatistics, September 2012. URL: http://dx.doi.org/10.1093/biostatistics/kxs033, doi:10.1093/biostatistics/kxs033.

[10] R. Dennis Cook. Detection of Influential Observation in Linear Regression. Technometrics, February 1977.

[11] Richard Bourgon, Robert Gentleman, and Wolfgang Huber. Independent filtering increases detection
power for high-throughput experiments. PNAS, 107(21):9546–9551, 2010. URL: http://www.pnas.org/content/107/21/9546.long.

[12] Yoav Benjamini and Yosef Hochberg. Controlling the false discovery rate: a practical and powerful
approach to multiple testing. Journal of the Royal Statistical Society B, 57:289–300, 1995.

[13] T. Schweder and E. Spjotvoll. Plots of P-values to evaluate many tests simultaneously. Biometrika,
69:493–502, 1982. doi:10.1093/biomet/69.3.493.